# 🧠 Deep Learning Model V3 — Root Cause Fixes
### Changes from V2:
1. **5x smaller model** (~800K vs 3.8M params) — directly combats overfitting
2. **Asymmetric loss** — penalizes under-prediction 1.5x more than over-prediction
3. **Deep Ensemble** — 3 models with different seeds, averaged predictions
4. **SWA** — Stochastic Weight Averaging for better generalization
5. **Bias correction** — post-processing calibration from validation set
6. **Scaling augmentation** — random magnitude warping during training
7. **OneCycleLR** — more aggressive learning rate schedule


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, json, time, math, copy, numpy as np, joblib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings; warnings.filterwarnings('ignore')

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)): return int(obj)
        if isinstance(obj, (np.floating,)): return float(obj)
        if isinstance(obj, np.ndarray): return obj.tolist()
        return super().default(obj)

BASE_DIR  = '/content/drive/MyDrive/Electricity_Load_Forecast'
MODEL_DIR = os.path.join(BASE_DIR, 'models')
V3_DIR    = os.path.join(MODEL_DIR, 'v3')
os.makedirs(V3_DIR, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
config        = joblib.load(os.path.join(MODEL_DIR, 'config.pkl'))
target_scaler = joblib.load(os.path.join(MODEL_DIR, 'target_scaler.pkl'))
X_train_w = np.load(os.path.join(MODEL_DIR, 'X_train_w.npy'))
y_train_w = np.load(os.path.join(MODEL_DIR, 'y_train_w.npy'))
X_val_w   = np.load(os.path.join(MODEL_DIR, 'X_val_w.npy'))
y_val_w   = np.load(os.path.join(MODEL_DIR, 'y_val_w.npy'))
X_test_w  = np.load(os.path.join(MODEL_DIR, 'X_test_w.npy'))
y_test_w  = np.load(os.path.join(MODEL_DIR, 'y_test_w.npy'))
N_FEATURES = config['N_FEATURES']
INPUT_LEN  = config['INPUT_LEN']
OUTPUT_LEN = config['OUTPUT_LEN']
print(f'Features={N_FEATURES}, In={INPUT_LEN}h, Out={OUTPUT_LEN}h')
print(f'Train={X_train_w.shape}, Val={X_val_w.shape}, Test={X_test_w.shape}')

In [ ]:
HP = {
    'batch_size': 64, 'lr': 5e-4, 'weight_decay': 5e-3,
    'max_epochs': 60, 'patience': 15, 'grad_clip': 0.5,
    'dropout': 0.35, 'noise_std': 0.03,
    'mixup_alpha': 0.3, 'scale_aug': 0.05,
    'conv_filters': 48, 'lstm_hidden': 128, 'n_heads': 4,
    'huber_delta': 1.0, 'under_weight': 1.5,
    'peak_weight': 2.5, 'peak_fraction': 0.2,
    'swa_start_frac': 0.7, 'swa_lr': 1e-5,
    'n_ensemble': 3, 'seeds': [42, 123, 456],
}
print('Hyperparameters:', json.dumps(HP, indent=2))

In [ ]:
def make_loader(X, y, bs, shuffle=True):
    ds = TensorDataset(torch.tensor(X, dtype=torch.float32),
                       torch.tensor(y, dtype=torch.float32))
    return DataLoader(ds, batch_size=bs, shuffle=shuffle,
                      num_workers=2, pin_memory=True, drop_last=shuffle)

train_loader = make_loader(X_train_w, y_train_w, HP['batch_size'])
val_loader   = make_loader(X_val_w, y_val_w, HP['batch_size'], shuffle=False)
test_loader  = make_loader(X_test_w, y_test_w, HP['batch_size'], shuffle=False)
print(f'Batches — Train:{len(train_loader)}, Val:{len(val_loader)}, Test:{len(test_loader)}')

In [ ]:
# ═══════════════════════════════════════
#  V3 ARCHITECTURE — Lighter & better
# ═══════════════════════════════════════
class LightCNN(nn.Module):
    """Two-branch CNN (k=3, k=7) instead of three-branch."""
    def __init__(self, ch, dropout=0.15):
        super().__init__()
        self.conv3 = nn.Conv1d(ch, ch, 3, padding=1)
        self.conv7 = nn.Conv1d(ch, ch, 7, padding=3)
        self.bn    = nn.BatchNorm1d(ch * 2)
        self.proj  = nn.Conv1d(ch * 2, ch, 1)
        self.bn2   = nn.BatchNorm1d(ch)
        self.drop  = nn.Dropout(dropout)

    def forward(self, x):
        c3 = F.gelu(self.conv3(x))
        c7 = F.gelu(self.conv7(x))
        out = F.gelu(self.bn(torch.cat([c3, c7], dim=1)))
        return self.drop(F.gelu(self.bn2(self.proj(out))))


class LoadForecastV3(nn.Module):
    """Lighter model: per-timestep decoder avoids information bottleneck.
    Input(B,168,F) → CNN → BiLSTM → SelfAttn → FFN → Decoder(B,168,1) → (B,168)"""
    def __init__(self, n_features, seq_len, pred_len,
                 conv_filters=48, lstm_hidden=128, n_heads=4,
                 dropout=0.35, noise_std=0.03):
        super().__init__()
        self.noise_std = noise_std
        d = lstm_hidden * 2

        self.input_proj = nn.Sequential(
            nn.Linear(n_features, conv_filters),
            nn.LayerNorm(conv_filters), nn.GELU(), nn.Dropout(dropout * 0.5))

        self.cnn = LightCNN(conv_filters, dropout * 0.5)

        self.lstm = nn.LSTM(conv_filters, lstm_hidden,
                            batch_first=True, bidirectional=True)
        self.ln1  = nn.LayerNorm(d)
        self.drop = nn.Dropout(dropout)

        self.attn    = nn.MultiheadAttention(d, n_heads, dropout=dropout, batch_first=True)
        self.ln2     = nn.LayerNorm(d)
        self.ffn     = nn.Sequential(nn.Linear(d, d*2), nn.GELU(),
                                     nn.Dropout(dropout), nn.Linear(d*2, d))
        self.ln3     = nn.LayerNorm(d)

        # Per-timestep decoder: each LSTM position → 1 output value
        self.decoder = nn.Sequential(
            nn.Linear(d, d // 2), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(d // 2, 1))
        self.output_bias = nn.Parameter(torch.zeros(pred_len))

    def forward(self, x):
        if self.training and self.noise_std > 0:
            x = x + torch.randn_like(x) * self.noise_std
        x = self.input_proj(x)                    # (B,168,48)
        x = self.cnn(x.permute(0,2,1)).permute(0,2,1)  # CNN
        h, _ = self.lstm(x)                       # (B,168,256)
        h = self.drop(self.ln1(h))
        a, _ = self.attn(h, h, h)
        h = self.ln2(h + a)
        h = self.ln3(h + self.ffn(h))
        return self.decoder(h).squeeze(-1) + self.output_bias  # (B,168)

# Quick param count
_m = LoadForecastV3(N_FEATURES, INPUT_LEN, OUTPUT_LEN,
                    HP['conv_filters'], HP['lstm_hidden'], HP['n_heads'],
                    HP['dropout'], HP['noise_std'])
print(f'V3 params: {sum(p.numel() for p in _m.parameters()):,}')
del _m

In [ ]:
# ═══════════════════════════════════════
#  ASYMMETRIC PEAK-AWARE LOSS
# ═══════════════════════════════════════
class AsymmetricPeakLoss(nn.Module):
    """Huber + asymmetric penalty (under-prediction costs more) + peak weighting."""
    def __init__(self, delta=1.0, under_weight=1.5, peak_weight=2.5, peak_fraction=0.2):
        super().__init__()
        self.huber = nn.HuberLoss(delta=delta, reduction='none')
        self.uw = under_weight
        self.pw = peak_weight
        self.pf = peak_fraction

    def forward(self, pred, target):
        loss = self.huber(pred, target)
        # Asymmetric: under-prediction penalised more
        aw = 1.0 + (self.uw - 1.0) * (pred < target).float()
        # Peak weighting
        k  = max(1, int(target.shape[1] * self.pf))
        th = torch.topk(target, k, dim=1).values[:, -1:].detach()
        pw = 1.0 + (self.pw - 1.0) * (target >= th).float()
        return (loss * aw * pw).mean()


def augment_batch(X, y, alpha=0.3, scale_range=0.05):
    """Mixup + random magnitude scaling."""
    if alpha > 0:
        lam = max(np.random.beta(alpha, alpha), 0.5)
        idx = torch.randperm(X.size(0), device=X.device)
        X = lam * X + (1-lam) * X[idx]
        y = lam * y + (1-lam) * y[idx]
    if scale_range > 0:
        s = 1.0 + (torch.rand(X.size(0),1,1, device=X.device)*2-1) * scale_range
        X = X * s
        y = y * s.squeeze(-1)
    return X, y

print('✅ Loss & augmentation defined')

In [ ]:
# ═══════════════════════════════════════
#  TRAIN ONE MODEL (with SWA)
# ═══════════════════════════════════════
def train_one(seed, HP, train_loader, val_loader, device, V3_DIR, idx):
    torch.manual_seed(seed); np.random.seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed(seed)

    model = LoadForecastV3(
        N_FEATURES, INPUT_LEN, OUTPUT_LEN,
        HP['conv_filters'], HP['lstm_hidden'], HP['n_heads'],
        HP['dropout'], HP['noise_std']
    ).to(device)

    crit = AsymmetricPeakLoss(HP['huber_delta'], HP['under_weight'],
                              HP['peak_weight'], HP['peak_fraction'])
    opt  = torch.optim.AdamW(model.parameters(), lr=HP['lr'], weight_decay=HP['weight_decay'])
    sched = torch.optim.lr_scheduler.OneCycleLR(
        opt, max_lr=HP['lr'], epochs=HP['max_epochs'],
        steps_per_epoch=len(train_loader), pct_start=0.1,
        anneal_strategy='cos', div_factor=10, final_div_factor=100)

    swa_start = int(HP['max_epochs'] * HP['swa_start_frac'])
    swa_model = AveragedModel(model)

    hist = {'train': [], 'val': [], 'lr': []}
    best_vl, patience_ctr, best_state = float('inf'), 0, None

    print(f"\n{'─'*45}")
    print(f"  Model {idx+1}/{HP['n_ensemble']}  seed={seed}")
    print(f"{'─'*45}")

    for ep in range(HP['max_epochs']):
        t0 = time.time()
        model.train(); losses = []
        for X_b, y_b in train_loader:
            X_b, y_b = X_b.to(device), y_b.to(device)
            X_b, y_b = augment_batch(X_b, y_b, HP['mixup_alpha'], HP['scale_aug'])
            opt.zero_grad()
            loss = crit(model(X_b), y_b)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), HP['grad_clip'])
            opt.step(); sched.step()
            losses.append(loss.item())

        if ep >= swa_start:
            swa_model.update_parameters(model)

        model.eval(); vl = []
        with torch.no_grad():
            for X_b, y_b in val_loader:
                vl.append(crit(model(X_b.to(device)), y_b.to(device)).item())

        tl = float(np.mean(losses)); vl = float(np.mean(vl))
        lr = float(opt.param_groups[0]['lr'])
        hist['train'].append(tl); hist['val'].append(vl); hist['lr'].append(lr)

        tag = ''
        if vl < best_vl:
            best_vl, patience_ctr = vl, 0
            best_state = copy.deepcopy(model.state_dict())
            tag = ' ✅'
        else:
            patience_ctr += 1

        if (ep+1) % 5 == 0 or tag:
            print(f'  Ep {ep+1:3d}/{HP["max_epochs"]} | '
                  f'T:{tl:.4f} V:{vl:.4f} LR:{lr:.1e} {time.time()-t0:.1f}s{tag}')

        if patience_ctr >= HP['patience']:
            print(f'  ⛔ Early stop ep {ep+1}'); break

    model.load_state_dict(best_state)
    swa_model.update_parameters(model)
    try: update_bn(train_loader, swa_model, device=device)
    except: pass

    torch.save(model.state_dict(), os.path.join(V3_DIR, f'model_{idx}_s{seed}.pt'))
    print(f'  ✅ Done (best val={best_vl:.4f})')
    return model, swa_model, hist, best_vl

In [ ]:
# ═══════════════════════════════════════
#  TRAIN ENSEMBLE
# ═══════════════════════════════════════
models, swa_models, histories, val_losses = [], [], [], []
for i, seed in enumerate(HP['seeds'][:HP['n_ensemble']]):
    m, sm, h, vl = train_one(seed, HP, train_loader, val_loader, device, V3_DIR, i)
    models.append(m); swa_models.append(sm)
    histories.append(h); val_losses.append(vl)

print(f"\n{'='*50}")
print(f'  ENSEMBLE DONE — val losses: {[round(v,4) for v in val_losses]}')
print(f'  Average val loss: {np.mean(val_losses):.4f}')
print(f"{'='*50}")

In [ ]:
fig, axes = plt.subplots(1, HP['n_ensemble'], figsize=(6*HP['n_ensemble'], 5))
if HP['n_ensemble'] == 1: axes = [axes]
for i, (h, ax) in enumerate(zip(histories, axes)):
    ax.plot(h['train'], label='Train', color='#2196F3')
    ax.plot(h['val'],   label='Val',   color='#FF5722')
    ax.set_title(f'Model {i+1} (seed={HP["seeds"][i]})')
    ax.set_xlabel('Epoch'); ax.set_ylabel('Loss')
    ax.legend(); ax.grid(True, alpha=0.3)
plt.suptitle('V3 Ensemble Training', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(V3_DIR, 'training_history_v3.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  EVALUATION
# ═══════════════════════════════════════
def predict_all(model, loader):
    model.eval(); ps = []
    with torch.no_grad():
        for X_b, _ in loader:
            ps.append(model(X_b.to(device)).cpu().numpy())
    return np.concatenate(ps)

def ensemble_predict(models, loader):
    return np.mean([predict_all(m, loader) for m in models], axis=0)

def inv(y): return target_scaler.inverse_transform(y.reshape(-1,1)).reshape(y.shape)

def metrics(y_true, y_pred):
    yt, yp = inv(y_true), inv(y_pred)
    mae  = float(mean_absolute_error(yt.flat, yp.flat))
    rmse = float(np.sqrt(mean_squared_error(yt.flat, yp.flat)))
    mask = yt.flat[:] != 0
    mape = float(np.mean(np.abs((yt.flat[mask]-yp.flat[mask])/yt.flat[mask]))*100)
    pk   = [float(np.abs(yt[i,np.argmax(yt[i])]-yp[i,np.argmax(yt[i])])) for i in range(len(yt))]
    return {'MAE':round(mae,2),'RMSE':round(rmse,2),'MAPE':round(mape,2),'Peak_MAE':round(float(np.mean(pk)),2)}

# Individual model results
print('Individual models (test):')
for i, m in enumerate(models):
    p = predict_all(m, test_loader)
    print(f'  Model {i+1}: {metrics(y_test_w[:len(p)], p)}')

# Ensemble
yp_val_ens  = ensemble_predict(models, val_loader)
yp_test_ens = ensemble_predict(models, test_loader)
ens_val  = metrics(y_val_w[:len(yp_val_ens)],  yp_val_ens)
ens_test = metrics(y_test_w[:len(yp_test_ens)], yp_test_ens)

print(f"\n{'='*40}\nEnsemble — Val:  {ens_val}")
print(f"Ensemble — Test: {ens_test}\n{'='*40}")

In [ ]:
# ═══════════════════════════════════════
#  BIAS CORRECTION (from validation set)
# ═══════════════════════════════════════
vp_inv = inv(yp_val_ens)
vt_inv = inv(y_val_w[:len(yp_val_ens)])
bias_per_hour = np.mean(vt_inv - vp_inv, axis=0)  # (168,)
print(f'Mean additive bias: {np.mean(bias_per_hour):.1f} MW')

# Apply to test
tp_inv = inv(yp_test_ens)
tt_inv = inv(y_test_w[:len(yp_test_ens)])
tp_corr = tp_inv + bias_per_hour

mae_c  = float(mean_absolute_error(tt_inv.flat, tp_corr.flat))
rmse_c = float(np.sqrt(mean_squared_error(tt_inv.flat, tp_corr.flat)))
mask   = tt_inv.flat[:] != 0
mape_c = float(np.mean(np.abs((tt_inv.flat[mask]-tp_corr.flat[mask])/tt_inv.flat[mask]))*100)
pk_c   = [float(np.abs(tt_inv[i,np.argmax(tt_inv[i])]-tp_corr[i,np.argmax(tt_inv[i])])) for i in range(len(tt_inv))]
corr_test = {'MAE':round(mae_c,2),'RMSE':round(rmse_c,2),'MAPE':round(mape_c,2),'Peak_MAE':round(float(np.mean(pk_c)),2)}

print(f"\n{'='*40}\nBias-Corrected Ensemble — Test: {corr_test}\n{'='*40}")

In [ ]:
# ═══════════════════════════════════════
#  HORIZON-WISE ERROR
# ═══════════════════════════════════════
horizons = [1, 24, 72, 168]
hmets = {}
for h in horizons:
    i = h - 1
    m_h = float(mean_absolute_error(tt_inv[:,i], tp_corr[:,i]))
    r_h = float(np.sqrt(mean_squared_error(tt_inv[:,i], tp_corr[:,i])))
    mk  = tt_inv[:,i] != 0
    mp_h = float(np.mean(np.abs((tt_inv[:,i][mk]-tp_corr[:,i][mk])/tt_inv[:,i][mk]))*100)
    hmets[h] = {'MAE':round(m_h,2),'RMSE':round(r_h,2),'MAPE':round(mp_h,2)}
print(f"{'Hour':>6} {'MAE':>10} {'RMSE':>10} {'MAPE%':>10}")
for h,m in hmets.items(): print(f'{h:>6} {m["MAE"]:>10} {m["RMSE"]:>10} {m["MAPE"]:>10}')

mph = [float(mean_absolute_error(tt_inv[:,h], tp_corr[:,h])) for h in range(OUTPUT_LEN)]
plt.figure(figsize=(12,5))
plt.plot(range(1,OUTPUT_LEN+1), mph, 'b-', lw=1.5)
for h in horizons:
    plt.axvline(h, color='red', ls='--', alpha=0.5)
    plt.annotate(f'h={h}', (h, mph[h-1]), fontsize=9, color='red')
plt.xlabel('Horizon (h)'); plt.ylabel('MAE (MW)')
plt.title('Horizon-Wise MAE — V3 Corrected'); plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(V3_DIR, 'horizon_wise_v3.png'), dpi=150)
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  FULL MODEL COMPARISON
# ═══════════════════════════════════════
with open(os.path.join(MODEL_DIR, 'baseline_metrics.json')) as f:
    bl = json.load(f)
v1, v2 = {}, {}
try:
    with open(os.path.join(MODEL_DIR, 'dl_metrics.json')) as f:
        raw = json.load(f)
    v1 = raw.get('CNN-BiLSTM-Attn', {'val':raw.get('val',{}),'test':raw.get('test',{})})
except: pass
try:
    with open(os.path.join(MODEL_DIR,'v2','dl_metrics_v2.json')) as f:
        raw2 = json.load(f)
    v2 = {'val':raw2.get('val',{}),'test':raw2.get('test',{})}
except: pass

all_m = {'Persistence': bl['Persistence'], 'XGBoost': bl['XGBoost']}
if v1: all_m['DL V1'] = v1
if v2: all_m['DL V2'] = v2
all_m['DL V3 Ensemble'] = {'val': ens_val, 'test': ens_test}
all_m['DL V3 Corrected'] = {'test': corr_test}

print(f"\n{'='*75}")
print(f'   FULL MODEL COMPARISON')
print(f"{'='*75}")
for sp in ['val','test']:
    print(f"\n--- {sp.upper()} ---")
    print(f"{'Model':<22} {'MAE':>8} {'RMSE':>8} {'MAPE%':>8} {'PeakMAE':>10}")
    print('-'*62)
    for n,m in all_m.items():
        if sp in m and m[sp]:
            t=m[sp]
            print(f"{n:<22} {t['MAE']:>8} {t['RMSE']:>8} {t['MAPE']:>8} {t['Peak_MAE']:>10}")

# Improvement vs XGBoost
xgb = bl['XGBoost']['test']
print(f"\n{'='*55}")
print(f'   V3 IMPROVEMENT vs XGBoost')
print(f"{'='*55}")
for label, mt in [('V3 Ensemble', ens_test), ('V3 Corrected', corr_test)]:
    print(f'\n  {label}:')
    for k in ['MAE','RMSE','MAPE','Peak_MAE']:
        xv, v3v = xgb[k], mt[k]
        pct = (xv - v3v) / xv * 100
        arr = '✅↓' if pct > 0 else '❌↑'
        print(f'    {k:10s}: XGB={xv:>8} → V3={v3v:>8}  ({pct:+.1f}%) {arr}')

# Bar chart
fig, axes = plt.subplots(1, 4, figsize=(22, 5))
mns = [n for n in all_m if 'test' in all_m[n] and all_m[n]['test']]
colors = ['#e74c3c','#3498db','#95a5a6','#f39c12','#2ecc71','#1abc9c']
for ax, mk in zip(axes, ['MAE','RMSE','MAPE','Peak_MAE']):
    vals = [all_m[n]['test'][mk] for n in mns]
    bars = ax.bar(range(len(mns)), vals, color=colors[:len(mns)], edgecolor='white')
    ax.set_title(mk, fontweight='bold'); ax.set_ylabel(mk)
    ax.set_xticks(range(len(mns))); ax.set_xticklabels(mns, rotation=25, ha='right', fontsize=8)
    for b,v in zip(bars,vals): ax.text(b.get_x()+b.get_width()/2, b.get_height()+0.5, f'{v}', ha='center', fontsize=8)
    ax.grid(True, alpha=0.2, axis='y')
plt.suptitle('All Models Comparison — Test Set', fontsize=14, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(V3_DIR, 'model_comparison_v3.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  SAVE ALL ARTIFACTS
# ═══════════════════════════════════════
import pandas as pd
v3_full = {
    'ensemble_val': ens_val, 'ensemble_test': ens_test,
    'corrected_test': corr_test,
    'horizon_wise': {str(k):v for k,v in hmets.items()},
    'individual_val_losses': [round(float(v),4) for v in val_losses],
    'hyperparameters': HP,
    'bias_per_hour_mean': round(float(np.mean(bias_per_hour)),2),
}
with open(os.path.join(V3_DIR, 'dl_metrics_v3.json'), 'w') as f:
    json.dump(v3_full, f, indent=2, cls=NumpyEncoder)

for i,h in enumerate(histories):
    pd.DataFrame(h).to_csv(os.path.join(V3_DIR, f'history_model{i}.csv'), index=False)

comp_save = {k:v for k,v in all_m.items() if 'test' in v and v['test']}
with open(os.path.join(V3_DIR, 'all_model_comparison_v3.json'), 'w') as f:
    json.dump(comp_save, f, indent=2, cls=NumpyEncoder)

np.save(os.path.join(V3_DIR, 'bias_per_hour.npy'), bias_per_hour)
print('\n✅ All V3 artifacts saved to models/v3/')

In [ ]:
# Sample predictions
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
n = len(yp_test_ens)
for idx, ax in enumerate(axes.flat):
    si = idx * (n // 4)
    ax.plot(tt_inv[si], 'k-', lw=2, label='Actual')
    ax.plot(tp_corr[si], 'g--', lw=1.5, label='V3 Corrected')
    ax.plot(tp_inv[si], 'b:', lw=1, alpha=0.5, label='V3 Raw')
    ax.set_title(f'Test #{si}'); ax.set_xlabel('Hour'); ax.set_ylabel('MW')
    ax.legend(); ax.grid(True, alpha=0.3)
plt.suptitle('V3 Predictions (Corrected vs Raw vs Actual)', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(V3_DIR, 'sample_predictions_v3.png'), dpi=150, bbox_inches='tight')
plt.show()
print('🎉 V3 complete!')